Set your parameters

In [1]:
# What is your major?
MAJOR_SUPPORT = "CISC"
# Do you need FE courses recommendation?
FE_SUPPORT = True
# What is the academic year now?
YEAR_SUPPORT = "2024a"

In [17]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.expand_frame_repr', False)

Convert your timetable into special data structure. You can use AI tools like GPT-4 or Claude-3.5 to accomplish this. In this case, you may need to use the following prompt:
```
my_timetable = [
{
"course_code": "GEST1009",
"day": "MON",
"start_time": "08:30",
"end_time": "09:45",
"type": "Lecture"
},
{
"course_code": "GEST1009",
"day": "THU",
"start_time": "08:30",
"end_time": "09:45",
"type": "Lecture"
},
{
"course_code": "CISC1002",
"day": "MON",
"start_time": "11:30",
"end_time": "12:45",
"type": "Lecture"
},
# ... (continue for all other classes)
]

Convert all the data from the timetable image into the format shown above.
```

In [2]:
timetable = [
    {
        "course_code": "GEST1009",
        "day": "MON",
        "start_time": "08:30",
        "end_time": "09:45",
        "type": "Lecture"
    },
    {
        "course_code": "GEST1009",
        "day": "THU",
        "start_time": "08:30",
        "end_time": "09:45",
        "type": "Lecture"
    },
    {
        "course_code": "CISC1002",
        "day": "MON",
        "start_time": "11:30",
        "end_time": "12:45",
        "type": "Lecture"
    },
    {
        "course_code": "CISC1002",
        "day": "THU",
        "start_time": "11:30",
        "end_time": "12:45",
        "type": "Lecture"
    },
    {
        "course_code": "CISC1002",
        "day": "FRI",
        "start_time": "10:00",
        "end_time": "11:15",
        "type": "Lab/Tutorial"
    },
    {
        "course_code": "GEGA1000",
        "day": "TUE",
        "start_time": "14:30",
        "end_time": "15:45",
        "type": "Lecture"
    },
    {
        "course_code": "GEGA1000",
        "day": "FRI",
        "start_time": "14:30",
        "end_time": "15:45",
        "type": "Lecture"
    },
    {
        "course_code": "CISC2001",
        "day": "MON",
        "start_time": "16:00",
        "end_time": "17:15",
        "type": "Lecture"
    },
    {
        "course_code": "CISC2001",
        "day": "THU",
        "start_time": "16:00",
        "end_time": "17:15",
        "type": "Lecture"
    },
    {
        "course_code": "CISC2001",
        "day": "FRI",
        "start_time": "17:30",
        "end_time": "18:45",
        "type": "Lab/Tutorial"
    },
    {
        "course_code": "CISC2003",
        "day": "TUE",
        "start_time": "16:00",
        "end_time": "17:15",
        "type": "Lecture"
    },
    {
        "course_code": "CISC2003",
        "day": "WED",
        "start_time": "16:00",
        "end_time": "17:15",
        "type": "Lab/Tutorial"
    },
    {
        "course_code": "CISC2003",
        "day": "FRI",
        "start_time": "16:00",
        "end_time": "17:15",
        "type": "Lecture"
    },
    {
        "course_code": "HONR2003",
        "day": "WED",
        "start_time": "18:30",
        "end_time": "21:15",
        "type": "Lecture"
    }
]

In [3]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

In [4]:
cm = pd.read_csv("2024a/cm.csv").dropna(axis=0)
ge = pd.read_csv("2024a/ge.csv").dropna(axis=0)

In [5]:
if FE_SUPPORT == True:
    df = pd.concat([cm, ge], ignore_index=True)
else:
    cm = cm[cm['course_name'].str.startswith(f'{MAJOR_SUPPORT}', na=False)]
    df = pd.concat([cm, ge], ignore_index=True)

In [6]:
grade_order = {
    'A+': 12, 'A': 11, 'A-': 10,
    'B+': 9, 'B': 8, 'B-': 7,
    'C+': 6, 'C': 5, 'C-': 4,
    'D+': 3, 'D': 2, 'D-': 1,
    'F': 0
}

def grade_to_numeric(grade):
    return grade_order.get(grade, -1) 

df = df.sort_values(
    by=['overall_grade', 'course_name', 'name'],
    key=lambda x: x.map(lambda y: grade_to_numeric(y) if x.name == 'overall_grade' else y),
    ascending=[False, True, True]
)

In [7]:
def parse_time(time_str):
    return datetime.strptime(time_str, '%H:%M')

def check_conflict(existing_class, new_class):
    if existing_class['day'] != new_class['day']:
        return False
    
    existing_start = parse_time(existing_class['start_time'])
    existing_end = parse_time(existing_class['end_time'])
    new_start = parse_time(new_class['start_time'])
    new_end = parse_time(new_class['end_time'])
    
    return (existing_start <= new_start < existing_end) or \
           (existing_start < new_end <= existing_end) or \
           (new_start <= existing_start and existing_end <= new_end)

def parse_course_time(time_str):
    time_slots = []
    for slot in time_str.split('&'):
        day, time_range = slot.strip().split(' ', 1)
        start_time, end_time = time_range.split('-')
        time_slots.append({
            'day': day,
            'start_time': start_time,
            'end_time': end_time
        })
    return time_slots

def has_conflict_with_timetable(course_time, my_timetable):
    course_slots = parse_course_time(course_time)
    for new_slot in course_slots:
        for existing_class in my_timetable:
            if check_conflict(existing_class, new_slot):
                return True
    return False

In [19]:
def show(df, timetable):
    non_conflicting_courses = df[~df['time'].apply(lambda x: has_conflict_with_timetable(x, timetable))]
    print(non_conflicting_courses[:30].to_string(index=False))

In [20]:
show(df, timetable)

course_name                 name  section overall_grade grade difficulty useful  comments                                                time
   ACCT3006        YUEN CHUN YIP        1             A     A          A      A         3                                     WED 10:00-11:15
   APAC4005         ZHOU YINNING        1             A     A          A      A         2                   MON 14:30-15:45 & THU 14:30-15:45
   CISC3001              WU YUAN        1             A     A          A      A         3 MON 17:30-18:45 & THU 17:30-18:45 & WED 10:00-11:15
   COMM2012          WANG HAIYAN        1             A     A          A      A         7                   FRI 11:30-12:45 & TUE 11:30-12:45
   CPED1001      CHAU HONG CHENG       22             A     A          A      A         4                                     FRI 11:30-13:15
   CPED1001      WONG SOI CHEONG       27             A     A          A      A         3                                     FRI 11:30-13:15
   FIN